# Builder Synth v1 — Test & Build Notebook

**Purpose:** Quick API testing for `builder_synth_v1`

Test corpus:
- **Multi-file projects:** `C-Programs/test/` (t01–t12)
- **Single-file programs:** `C-Programs/simple_programs/`

Build matrix per job: **4 optimizations × 3 variants = 12 cells**
- Optimizations: O0, O1, O2, O3
- Variants: debug (with `-g`), release, stripped

## Setup and Configuration

Import libraries and configure paths.

In [1]:
import requests
import json
from pathlib import Path
import time

# API configuration
API_BASE = "http://localhost:8080/builder"
API_SYNTHETIC = f"{API_BASE}/synthetic"

# Corpus paths
TEST_DIR = Path(r"C:\Users\nico_\Documents\UNI\Thesis\Source\reforge\C-Programs\test")
SIMPLE_DIR = Path(r"C:\Users\nico_\Documents\UNI\Thesis\Source\reforge\C-Programs\simple_programs")

print("✓ Configuration loaded")
print(f"  API:     {API_BASE}")
print(f"  Test:    {TEST_DIR}")
print(f"  Simple:  {SIMPLE_DIR}")

✓ Configuration loaded
  API:     http://localhost:8080/builder
  Test:    C:\Users\nico_\Documents\UNI\Thesis\Source\reforge\C-Programs\test
  Simple:  C:\Users\nico_\Documents\UNI\Thesis\Source\reforge\C-Programs\simple_programs


## API Helper Functions

Core functions to interact with the builder API.

In [2]:
def submit_build(name, test_category, files=None, source_code=None, optimizations=None, target=None):
    """
    Submit a build job to the API.
    
    Args:
        name: Unique identifier for this test case
        test_category: Category tag
        files: List of {filename, content} dicts (for multi-file)
        source_code: Single C source string (for single-file)
        optimizations: List of O-levels (default: all)
        target: Optional {optimization, variant} for single cell rebuild
    
    Returns:
        Response JSON with job_id, name, status, message
    """
    payload = {
        "name": name,
        "test_category": test_category,
        "language": "c",
        "optimizations": optimizations or ["O0", "O1", "O2", "O3"],
    }
    
    if files:
        payload["files"] = files
    elif source_code:
        payload["source_code"] = source_code
    else:
        raise ValueError("Must provide either 'files' or 'source_code'")
    
    if target:
        payload["target"] = target
    
    try:
        response = requests.post(API_SYNTHETIC, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f" Error submitting {name}: {e}")
        if hasattr(e.response, 'text'):
            print(f"   Response: {e.response.text}") #type: ignore
        return None


def get_job_status(job_id):
    """Get status of a job by UUID."""
    try:
        response = requests.get(f"{API_BASE}/job/{job_id}")
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f" Error fetching job {job_id}: {e}")
        return None


def get_build_by_name(name):
    """Get build details by test case name."""
    try:
        response = requests.get(f"{API_SYNTHETIC}/{name}")
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f" Error fetching {name}: {e}")
        return None


def delete_build(name):
    """Delete a build by name."""
    try:
        response = requests.delete(f"{API_SYNTHETIC}/{name}")
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f" Error deleting {name}: {e}")
        return None


def delete_all_builds():
    """Delete ALL synthetic builds (use with caution)."""
    try:
        response = requests.delete(API_SYNTHETIC)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f" Error deleting all builds: {e}")
        return None


print("✓ API helpers loaded")

✓ API helpers loaded


## Build All Multi-File Projects

Iterate through all test projects (t01–t12) and submit builds.

In [3]:
def build_all_multifile():
    """Build all multi-file test projects from the test corpus."""
    projects = sorted([d for d in TEST_DIR.iterdir() if d.is_dir()])
    
    results = []
    for project_dir in projects:
        manifest_path = project_dir / "manifest.json"
        if not manifest_path.exists():
            print(f"  Skipping {project_dir.name} (no manifest.json)")
            continue
        
        # Read manifest
        manifest = json.loads(manifest_path.read_text())
        name = manifest.get("name", project_dir.name)
        category = manifest.get("description", "test")
        
        # Read all source files
        files = []
        for src_file in manifest.get("files", []):
            src_path = project_dir / src_file
            if src_path.exists():
                files.append({
                    "filename": src_file,
                    "content": src_path.read_text()
                })
        
        if not files:
            print(f"  Skipping {name} (no source files)")
            continue
        
        # Submit
        print(f" Submitting {name} ({len(files)} files)...", end=" ")
        result = submit_build(name, category, files=files)
        
        if result:
            print(f"✓ {result['job_id']}")
            results.append(result)
        else:
            print(" FAILED")
    
    print(f"\n Submitted {len(results)}/{len(projects)} multi-file projects")
    return results

# Uncomment to run:
build_all_multifile()

 Submitting t01_crossfile_calls (4 files)... ✓ da1aef2c-50e0-412b-a8a1-1074dfd18389
 Submitting t02_shared_header_macros (4 files)... ✓ 75d3baf3-2a87-47c3-90f5-15f8f2c442ef
 Submitting t03_header_dominant (4 files)... ✓ 96b92b55-e632-49c0-90ba-20146c15daa6
 Submitting t04_static_dup_names (5 files)... ✓ d08e4a9f-b34e-4d49-b67e-a31d8da276f8
 Submitting t05_fptr_callbacks (4 files)... ✓ 1b03e08c-7a2c-4d5d-b580-e0bf62f89d44
 Submitting t06_recursion_inline (4 files)... ✓ f785df28-f803-4b96-85f6-c0a1092465f1
 Submitting t07_switch_parser (4 files)... ✓ d3af5c1c-7860-4b52-abb7-3ebc18e2acf7
 Submitting t08_loop_heavy (4 files)... ✓ 981460b3-7648-4ad3-ae4e-a86b21ba4700
 Submitting t09_string_format (4 files)... ✓ a887ea6a-ab89-4816-ae93-2e287ec7f6bc
 Submitting t10_math_libm (4 files)... ✓ e31feb92-268b-4081-9919-ab1cd117c442
 Submitting t11_mixed_stress (5 files)... ✓ 0fad0d42-0ec0-4977-b4ab-7002fb98e3fb
 Submitting t12_state_machine (4 files)... ✓ e6c9cd96-47af-4d85-9d41-ffacec5fe102

 Subm

[{'job_id': 'da1aef2c-50e0-412b-a8a1-1074dfd18389',
  'name': 't01_crossfile_calls',
  'status': 'QUEUED',
  'message': "Synthetic build queued for 't01_crossfile_calls' (4 file(s), 4 opt levels)"},
 {'job_id': '75d3baf3-2a87-47c3-90f5-15f8f2c442ef',
  'name': 't02_shared_header_macros',
  'status': 'QUEUED',
  'message': "Synthetic build queued for 't02_shared_header_macros' (4 file(s), 4 opt levels)"},
 {'job_id': '96b92b55-e632-49c0-90ba-20146c15daa6',
  'name': 't03_header_dominant',
  'status': 'QUEUED',
  'message': "Synthetic build queued for 't03_header_dominant' (4 file(s), 4 opt levels)"},
 {'job_id': 'd08e4a9f-b34e-4d49-b67e-a31d8da276f8',
  'name': 't04_static_dup_names',
  'status': 'QUEUED',
  'message': "Synthetic build queued for 't04_static_dup_names' (5 file(s), 4 opt levels)"},
 {'job_id': '1b03e08c-7a2c-4d5d-b580-e0bf62f89d44',
  'name': 't05_fptr_callbacks',
  'status': 'QUEUED',
  'message': "Synthetic build queued for 't05_fptr_callbacks' (4 file(s), 4 opt levels

## Build Single Multi-File Project

Build a specific multi-file project by name.

In [ ]:
def build_one_multifile(project_name):
    """
    Build a specific multi-file project.
    
    Args:
        project_name: Directory name (e.g., "t01_crossfile_calls")
    """
    project_dir = TEST_DIR / project_name
    if not project_dir.exists():
        print(f" Project not found: {project_name}")
        return None
    
    manifest_path = project_dir / "manifest.json"
    if not manifest_path.exists():
        print(f" No manifest.json in {project_name}")
        return None
    
    # Read manifest
    manifest = json.loads(manifest_path.read_text())
    name = manifest.get("name", project_name)
    category = manifest.get("description", "test")
    
    # Read all source files
    files = []
    for src_file in manifest.get("files", []):
        src_path = project_dir / src_file
        if src_path.exists():
            files.append({
                "filename": src_file,
                "content": src_path.read_text()
            })
    
    if not files:
        print(f" No source files found for {name}")
        return None
    
    print(f" Building {name} ({len(files)} files):")
    for f in files:
        print(f"   - {f['filename']}")
    
    result = submit_build(name, category, files=files)
    
    if result:
        print(f" Submitted: {result['job_id']}")
    
    return result

# Example:
# build_one_multifile("t01_crossfile_calls")

## Build All Single-File Programs

Build all single-file programs from `simple_programs/`.

In [4]:
def build_all_singlefile():
    """Build all single-file C programs from simple_programs/."""
    c_files = sorted(SIMPLE_DIR.glob("*.c"))
    
    results = []
    for c_file in c_files:
        name = c_file.stem
        source_code = c_file.read_text()
        
        print(f" Submitting {name}...", end=" ")
        result = submit_build(
            name=name,
            test_category="simple_programs",
            source_code=source_code
        )
        
        if result:
            print(f"✓ {result['job_id']}")
            results.append(result)
        else:
            print(" FAILED")
    
    print(f"\n Submitted {len(results)}/{len(c_files)} single-file programs")
    return results

# Uncomment to run:
build_all_singlefile()

 Submitting Calculator... ✓ d04b82aa-74b0-404e-a0f5-44e304ed0c97
 Submitting ContactManagementSystem... ✓ 0f620fa4-cd71-458f-97c3-a8e31436000b
 Submitting HangmanGame... ✓ b1cad998-6a61-4d15-8c10-2729dbc1f85b
 Submitting LibraryManagementSystem... ✓ 7b6dff31-3621-4c36-96ed-7d450397805b
 Submitting neg_inline_user... ✓ 8d9f11e9-3e6c-4bb6-9b22-cfbb946c72c0
 Submitting NumberGuess... ✓ d7a7b60c-54f9-4a1e-b82d-14a824964186
 Submitting s1_basic... ✓ a07c38f3-6d43-43c0-83d0-4f8557085af5
 Submitting s2_recursion... ✓ 83169ba5-5c35-4aa3-8c99-9c07128f2a7a
 Submitting s3_fptr... ✓ 497c34c1-0352-475e-8d4b-1e46187fd983
 Submitting s4_switch... ✓ c1c2a529-46c0-4ea1-a8da-1ec6f12880df
 Submitting s5_computed_goto... ✓ b0121e24-828d-437c-8d7f-7c7fe974987e
 Submitting StudentGradeBook... ✓ 69d18b44-02fd-4e90-8e33-87606228ece0

 Submitted 12/12 single-file programs


[{'job_id': 'd04b82aa-74b0-404e-a0f5-44e304ed0c97',
  'name': 'Calculator',
  'status': 'QUEUED',
  'message': "Synthetic build queued for 'Calculator' (1 file(s), 4 opt levels)"},
 {'job_id': '0f620fa4-cd71-458f-97c3-a8e31436000b',
  'name': 'ContactManagementSystem',
  'status': 'QUEUED',
  'message': "Synthetic build queued for 'ContactManagementSystem' (1 file(s), 4 opt levels)"},
 {'job_id': 'b1cad998-6a61-4d15-8c10-2729dbc1f85b',
  'name': 'HangmanGame',
  'status': 'QUEUED',
  'message': "Synthetic build queued for 'HangmanGame' (1 file(s), 4 opt levels)"},
 {'job_id': '7b6dff31-3621-4c36-96ed-7d450397805b',
  'name': 'LibraryManagementSystem',
  'status': 'QUEUED',
  'message': "Synthetic build queued for 'LibraryManagementSystem' (1 file(s), 4 opt levels)"},
 {'job_id': '8d9f11e9-3e6c-4bb6-9b22-cfbb946c72c0',
  'name': 'neg_inline_user',
  'status': 'QUEUED',
  'message': "Synthetic build queued for 'neg_inline_user' (1 file(s), 4 opt levels)"},
 {'job_id': 'd7a7b60c-54f9-4a1e

## Build Single-File Program

Build a specific single-file program.

In [ ]:
def build_one_singlefile(filename):
    """
    Build a specific single-file program.
    
    Args:
        filename: Name of the .c file (e.g., "Calculator.c")
    """
    c_file = SIMPLE_DIR / filename
    if not c_file.exists():
        print(f" File not found: {filename}")
        return None
    
    name = c_file.stem
    source_code = c_file.read_text()
    
    print(f" Building {name}...")
    print(f"   Source: {filename}")
    print(f"   Lines:  {len(source_code.splitlines())}")
    
    result = submit_build(
        name=name,
        test_category="simple_programs",
        source_code=source_code
    )
    
    if result:
        print(f" Submitted: {result['job_id']}")
    
    return result

# Example:
# build_one_singlefile("Calculator.c")

## Check Job Status

Query build status by job ID or name.

In [ ]:
# Check by job ID
job_id = "your-job-id-here"
# status = get_job_status(job_id)
# print(json.dumps(status, indent=2))

# Check by name
name = "t01_crossfile_calls"
# status = get_build_by_name(name)
# if status:
#     print(f"Name:     {status['name']}")
#     print(f"Status:   {status['status']}")
#     print(f"Category: {status['test_category']}")
#     print(f"Files:    {status['file_count']}")
#     print(f"SHA256:   {status['snapshot_sha256']}")
#     print(f"Binaries: {status['binary_count']}")
#     print(f"Created:  {status.get('created_at', 'N/A')}")

## Query Build Results

Get detailed build results including all binaries.

In [ ]:
def show_build_results(name):
    """Display detailed results for a completed build."""
    result = get_build_by_name(name)
    if not result:
        return
    
    print(f"{'='*60}")
    print(f"Build: {result['name']}")
    print(f"{'='*60}")
    print(f"Status:   {result['status']}")
    print(f"Category: {result['test_category']}")
    print(f"Files:    {result['file_count']}")
    print(f"Binaries: {result['binary_count']}")
    print(f"SHA-256:  {result['snapshot_sha256']}")
    print(f"Created:  {result.get('created_at', 'N/A')}")
    
    if result['binary_count'] > 0:
        print(f"\n{'Binaries:':60}")
        print(f"{'-'*60}")
        for binary in result['binaries']:
            opt = binary['optimization_level']
            var = binary['variant_type']
            size = binary['file_size']
            debug = '✓' if binary['has_debug_info'] else '✗'
            strip = '✓' if binary['is_stripped'] else '✗'
            
            print(f"{opt:3} / {var:8}  |  {size:8,} bytes  |  debug:{debug}  strip:{strip}")
            
            # Show ELF metadata if present
            if binary.get('elf_metadata'):
                elf = binary['elf_metadata']
                print(f"              ELF: {elf.get('elf_type', 'N/A')} / {elf.get('arch', 'N/A')}")
    
    return result

# Example:
# show_build_results("t01_crossfile_calls")

## Rebuild Specific Cell

Rebuild a single optimization/variant combination using the `target` parameter.

In [ ]:
def rebuild_cell(project_name, optimization, variant):
    """
    Rebuild a specific optimization/variant cell.
    
    Args:
        project_name: Multi-file project directory or single-file stem
        optimization: "O0", "O1", "O2", or "O3"
        variant: "debug", "release", or "stripped"
    """
    # Try multi-file first
    project_dir = TEST_DIR / project_name
    if project_dir.exists():
        manifest_path = project_dir / "manifest.json"
        if manifest_path.exists():
            manifest = json.loads(manifest_path.read_text())
            name = manifest.get("name", project_name)
            category = manifest.get("description", "test")
            
            files = []
            for src_file in manifest.get("files", []):
                src_path = project_dir / src_file
                if src_path.exists():
                    files.append({
                        "filename": src_file,
                        "content": src_path.read_text()
                    })
            
            print(f" Rebuilding {name} cell: {optimization}/{variant}")
            result = submit_build(
                name=name,
                test_category=category,
                files=files,
                target={"optimization": optimization, "variant": variant}
            )
            
            if result:
                print(f" Rebuild submitted: {result['job_id']}")
            return result
    
    # Try single-file
    c_file = SIMPLE_DIR / f"{project_name}.c"
    if c_file.exists():
        source_code = c_file.read_text()
        
        print(f" Rebuilding {project_name} cell: {optimization}/{variant}")
        result = submit_build(
            name=project_name,
            test_category="simple_programs",
            source_code=source_code,
            target={"optimization": optimization, "variant": variant}
        )
        
        if result:
            print(f" Rebuild submitted: {result['job_id']}")
        return result
    
    print(f" Project not found: {project_name}")
    return None

# Example:
# rebuild_cell("t01_crossfile_calls", "O2", "stripped")

## Cleanup Operations

Delete individual builds or all synthetic builds.

In [ ]:
# Delete a specific build
# name = "t01_crossfile_calls"
# result = delete_build(name)
# if result:
#     print(f" Deleted: {result['name']}")
#     print(f"   Binaries deleted: {result['binaries_deleted']}")

#  Delete ALL builds (use with caution!)
# result = delete_all_builds()
# if result:
#     print(f" Deleted all builds:")
#     print(f"   Synthetic builds: {result['synthetic_builds_deleted']}")
#     print(f"   Binaries:         {result['binaries_deleted']}")

## Quick Test Scenarios

Ready-to-run test scenarios.

In [ ]:
# Scenario 1: Build one test project and check results
# build_one_multifile("t05_fptr_callbacks")
# time.sleep(5)  # Wait for build to complete
# show_build_results("t05_fptr_callbacks")

# Scenario 2: Build one simple program with only O2 optimization
# result = submit_build(
#     name="Calculator_test",
#     test_category="simple_programs",
#     source_code=(SIMPLE_DIR / "Calculator.c").read_text(),
#     optimizations=["O2"]
# )
# print(f"Job ID: {result['job_id']}")

# Scenario 3: Rebuild just the stripped variant at O3
# rebuild_cell("t06_recursion_inline", "O3", "stripped")

# Scenario 4: Build all multi-file projects
# build_all_multifile()

# Scenario 5: Build all simple programs
# build_all_singlefile()